LP专题-04：广告流量分配：曝光和转化均衡 [](https://opt.aliyun.com/example/q6VfqijIRnyi/path/src-LP04_视频流量调控_solver_Python_api.ipynb)

![](LP04_视频流量调控.png)

In [87]:
from pprint import pprint

import docplex.mp.model as cpx
import pandas as pd

iIdx = [0, 1, 2]
jIdx = [0, 1]
ctr = {
    (0, 0): 0.52,
    (0, 1): 0.92,
    (1, 0): 0.31,
    (1, 1): 0.93,
    (2, 0): 0.82,
    (2, 1): 0.91,
}
vMin = {
    # i: 1 if i == 2 else 0 for i in iIdx
    0: 0,
    1: 0,
    2: 1,
}

model = cpx.Model(name="MIP Model")

# if x is Continuous
xArr = {
    (i, j): model.continuous_var(
        lb=0.0, ub=1, name="x_{0}_{1}".format(i, j)
    )
    for j in jIdx
    for i in iIdx
}

# <= constraints，小于等于
constraints = []
for i in iIdx:
    constraints.append(model.add_constraint(
        ct=model.sum(ctr[i, j] * xArr[i, j] for j in jIdx) >= vMin[i],
        ctname="constraint_0_{0}".format(i),
    ))
for j in jIdx:
    constraints.append(model.add_constraint(
        ct=model.sum(xArr[i, j] for i in iIdx) == 1,
        ctname="constraint_1_{0}".format(j),
    ))

objective = model.sum(ctr[i, j] * xArr[i, j] for i in iIdx for j in jIdx)

# for maximization
model.maximize(objective)

# solving with local cplex
model.solve()

pprint(xArr)
# output
df = pd.DataFrame.from_dict(xArr, orient="index", columns=["variable_object"])
df.index = pd.MultiIndex.from_tuples(df.index, names=["column_i", "column_j"])
df.reset_index(inplace=True)

# CPLEX
df["solution_value"] = df["variable_object"].apply(lambda item: item.solution_value)

df.drop(columns=["variable_object"], inplace=True)
# df.to_csv("./optimization_solution.csv")

df

{(0, 0): docplex.mp.Var(type=C,name='x_0_0',ub=1),
 (0, 1): docplex.mp.Var(type=C,name='x_0_1',ub=1),
 (1, 0): docplex.mp.Var(type=C,name='x_1_0',ub=1),
 (1, 1): docplex.mp.Var(type=C,name='x_1_1',ub=1),
 (2, 0): docplex.mp.Var(type=C,name='x_2_0',ub=1),
 (2, 1): docplex.mp.Var(type=C,name='x_2_1',ub=1)}


,column_i,column_j,solution_value
0,0,0,0.000000
1,1,0,0.000000
2,2,0,1.000000
3,0,1,0.000000
4,1,1,0.802198
5,2,1,0.197802
